In [ ]:
import pandas as pd
from PIL import Image
import numpy as np

In [ ]:
df = pd.read_csv('book_w_images.csv')

In [ ]:
# df = pd.read_csv('book_w_image_engineering_draft1.csv')

In [ ]:
df = df.drop(['genre_0', 'genre_1', 'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7', 'genre_8', 'genre_9', 'genre_10', 'genre_11', 
              'genre_12', 'genre_13', 'genre_14', 'genre_15', 'genre_16', 'genre_17', 'genre_18', 'genre_19'], axis = 1)

In [ ]:
df = df[df.pages > 0]
df = df[df.totalratings < 10000]

In [ ]:
df.head(2)

,author,bookformat,desc,img,isbn,isbn13,link,pages,rating,reviews,title,totalratings,cover_image
0,Robert Reed,Paperback,ship traveled universe longer crew recall true...,https://i.gr-assets.com/images/S/compressed.ph...,812566572,9.780000e+12,https://goodreads.com/book/show/100208.Marrow,512,3.82,110,Marrow,2051,cover_images/processed_image_111.jpg
1,"Wu Cheng'en,Arthur Waley,Hu Shih",Paperback,probably popular book history far east classic...,https://i.gr-assets.com/images/S/compressed.ph...,802130860,9.780000e+12,https://goodreads.com/book/show/100237.Monkey,306,4.02,552,Monkey: The Journey to the West,6147,cover_images/processed_image_139.jpg


In [ ]:
df = df.drop(['title', 'link', 'isbn13', 'isbn', 'img', 'desc', 'author'
            #   , 'cover_image'
              ], axis = 1)
# df = df.drop(['author', 'desc', 'title', 'img', 'isbn', 'isbn13', 'link','cover_image'
#             #   ], axis = 1)
# df = df.drop(['title', 'author', 'Unnamed: 0', 'desc'])

In [ ]:
df['numerical_representation'] = ''
path_column = 'cover_image'

df['color_hist'] = ''

In [ ]:
def numerize_images(df, path_column):
    for index, row in df.iterrows():
        image_path = row[path_column]

        try:
            # Image processing
            image = Image.open(image_path)
            
            # Convert the image to grayscale
            image = image.convert('L')

            # Resize the image to a consistent size (e.g., 100x100 pixels)
            image = image.resize((100, 100))

            # Convert the image to a numerical representation (pixel values)
            image_array = np.array(image).flatten()

            # Update DataFrame with numerical representation
            df.at[index, 'numerical_representation'] = list(image_array)
        except OSError as e:
            print(f'Failed to process image {index}. OSError: {e}')

# Example usage:
# Assuming 'image_path_column' is the column containing paths to JPEGs
# Assuming 'df' is your DataFrame
numerize_images(df, path_column)

In [ ]:
def compute_color_histogram(image):
    # Convert the image to RGB mode if not already
    image_rgb = image.convert('RGB')

    # Compute the color histogram
    histogram = np.array(image_rgb.histogram())

    # Normalize the histogram
    histogram = histogram / sum(histogram)

    return list(histogram)

def extract_color_histograms(df, path_column):
    # Create a list to store color histograms
    color_histograms = []

    for index, row in df.iterrows():
        image_path = row[path_column]

        try:
            # Image processing
            image = Image.open(image_path)

            # Compute color histogram
            color_histogram = compute_color_histogram(image)

            # Append the entire color histogram to the list
            color_histograms.append(color_histogram)
        except OSError as e:
            print(f'Failed to process image {index}. OSError: {e}')

    # Add the 'color_hist' column to the DataFrame
    df['color_hist'] = color_histograms

# Example usage:
# Assuming 'cover_image' is the column containing paths to JPEGs
# Assuming 'df' is your DataFrame
path_column = 'cover_image'
extract_color_histograms(df, path_column)

In [ ]:
df.head(1)

,bookformat,pages,rating,reviews,totalratings,cover_image,numerical_representation,color_hist
0,Paperback,512,3.82,110,2051,cover_images/processed_image_111.jpg,"[160, 186, 182, 182, 186, 182, 188, 185, 183, ...","[0.0006333333333333333, 0.0002666666666666667,..."


In [ ]:
df = df.drop(['cover_image'], axis = 1)

In [ ]:
# Flatten the columns with lists
df_flat = df.apply(lambda x: pd.Series(x['numerical_representation'] + x['color_hist']), axis=1)

# Concatenate the flattened columns with the original DataFrame
df_result = pd.concat([df, df_flat], axis=1)

# Drop the original list columns
df = df_result.drop(['numerical_representation', 'color_hist'], axis=1)

In [ ]:
# Specify the column to create dummies
column_to_dummy = 'bookformat'

# Create dummies for the specified column
dummies = pd.get_dummies(df[column_to_dummy], prefix = column_to_dummy)

# Concatenate the dummies with the original DataFrame, keeping other columns
df_with_dummies = pd.concat([df, dummies], axis = 1)

# Drop the original column for which dummies are created
df = df_with_dummies.drop(column_to_dummy, axis=1)

In [ ]:
df.columns = df.columns.astype(str)

In [ ]:
df.to_csv('book_df_final.csv')